# Text Generation by Analytics Vidhya
source: https://www.analyticsvidhya.com/blog/2018/03/text-generation-using-python-nlp/

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


## Loading Data

In [2]:
sent_df = pd.read_csv("sentiment_dataframe.csv")
sent_df.head()

,index,sentence,chapter,neg,neu,pos,compound,chapter_text,moving_avg_comp,chapter_author,chapter_text_num
0,0,PART ONE CONTAINING THE PAPERS OF A Are pas...,0,0.000,0.814,0.186,0.4939,Preface,0.080093,Neutral,NaN
1,1,Reason alone baptized?,0,0.500,0.500,0.000,-0.2500,Preface,0.084089,Neutral,NaN
2,2,Edward Young1 PREFACE PERHAPS it has sometime...,0,0.083,0.773,0.144,0.6486,Preface,0.084089,Neutral,NaN
3,3,Your life has perhaps brought you into touch w...,0,0.054,0.850,0.096,0.3612,Preface,0.084089,Neutral,NaN
4,4,Perhaps neither case applies to you and your l...,0,0.071,0.857,0.071,0.0000,Preface,0.089234,Neutral,NaN


chapter_text_num is NaN only for Preface, this is ok as this column was only created for graphical visualations. Basically just shortened chapter names to numbers. 

## Creating Word/Character Mappings

There are interesting pros and cons to word vs character mapping highlighted by this [lighttag.io blogpost](https://www.lighttag.io/blog/character-level-NLP/). <br>
### Advantages
- Character mapping is more general when discerning syntax. With word mapping, words like 3/4", csv w/, or 50k need to be specified. A regular model usually has the 10-30k most frequent words in its vocabulary, and so "unusual" looking words are interpreted poorly. Character mapping sees the input "as-is" and each word is equally strange. This is beneficial for poorly spelled, user-generated text as this approach is more generalized and robust. <br>
- Character level models 

Below we can visualize what is inside of our character mapping. I noticed some weird errors of sentences composed of only numbers and went back to filter them out. 

In [3]:
join_with = " "
text = join_with.join(sent_df["sentence"][sent_df["chapter_author"] == "Aesthetic"])
characters = sorted(list(set(text)))
#n_to_char = {n:char for n, char in enumerate(characters)}
#char_to_n = {char:n for n, char in enumerate(characters)}
text

'  1 DIAPSALMATA  ad se ipsum  Grandeur, savoir, renommé, Amitié, plaisir et bien, Tout n’est que vent, que fumée: Pour mieux dire, tout n’est rien.1  WHAT is a poet? An unhappy man who hides deep anguish in his heart, but whose lips are so formed that when the sigh and cry pass through them, it sounds like lovely music. His fate is like that of those unfortunates who were slowly tortured by a gentle fire in Phalaris’s bull; their cries could not reach the tyrant’s ears to cause him dismay, to him they sounded like sweet music.2 And people flock around the poet and say: ‘Sing again soon’ – that is, ‘May new sufferings torment your soul but your lips be fashioned as before, for the cry would only frighten us, but the music, that is blissful.’ And the critics come forward and say: ‘That’s the way, that’s how the rules of aesthetics say it should be done.’ Of course, a critic resembles a poet to a hair, except he has no anguish in his heart, no music on his lips. So I tell you, I would ra